In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U sentence-transformers 

In [ ]:
import json
import gzip
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import tqdm
import os
import pandas as pd
from sentence_transformers import util

paragraphs = set()

# Load Chemicals domain corpus
corpus_filepath = '/kaggle/input/industrial-chemical-domain-corpus/chemical_domain_corpus.csv'
paragraphs = set(pd.read_csv(corpus_filepath).context.values)
paragraphs = list(paragraphs)
paragraphs = [paragraph for paragraph in paragraphs if len(paragraph)>100]
paragraphs = [paragraph.replace("\t", " ").replace("\n", " ").strip() for paragraph in paragraphs if len(paragraph)>100 and ':' not in paragraph]
len(paragraphs)

In [ ]:
# No we load the model that is able to generate queries given a paragraph.
# This model was trained on the MS MARCO dataset, a dataset with 500k
# queries from Bing and the respective relevant passage
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model.eval()

#Select the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Parameters for generation
batch_size = 8 #Batch size
num_queries = 2 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query



In [ ]:
# Now for every paragraph in our corpus, we generate the queries
dataset = []
#with open('/kaggle/working/generated_queries.tsv', 'w') as fOut:
for start_idx in tqdm.trange(0, len(paragraphs), batch_size):
    sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
    inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length_query,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_queries)

    for idx, out in enumerate(outputs):
        query = tokenizer.decode(out, skip_special_tokens=True)
        para = sub_paragraphs[int(idx/num_queries)]
        query = query.replace("\t", " ").strip()
        para = para.replace("\t", " ").replace("\n", " ").strip()
        dataset.append({
            "question": query,
            "context": para
        })
            #fOut.write("{} | {}\n".format(, ))


In [ ]:
pd.DataFrame(dataset).to_csv('/kaggle/working/generated_queries.csv', index=False, encoding='utf-8-sig')